## Income Dynamics Lab Spring 2022
### Record linkage Jupyter Notebook
_Goal: Link SHG members data with Job Card Data_

This notebook links Self-Help Group (SHG) member data with Job Card data using fuzzy matching. The primary objective is to facilitate the identification of members across datasets. The project uses the `recordlinkage` Python library and Google Colab as the primary development environment.



In [ ]:
# Install necessary packages (local install)
!pip install recordlinkage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import the necessary packages (local install and some included with google colab)
import pandas as pd
import recordlinkage
from multiprocessing import Pool
import math,time,numpy,string
from google.colab import drive 
import glob
import jellyfish.cjellyfish

In [ ]:
# Set display options for Jupyter output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
from google.colab import data_table
data_table.enable_dataframe_formatter()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Set up the path, link with drive
drive.mount('/content/gdrive/')
gdrive_path = "/content/gdrive/Shareddrives/Income_Dynamics_2022/FuzzyMatchesData/"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
# Get subset of Bihar SHG data

# Read SHG_Details.csv
shg_details = "SHG_Details.csv"
# df_shg_detail = pd.read_csv(gdrive_path + shg_details, encoding = 'utf-16')
df_shg_detail = pd.read_csv("/content/gdrive/Shareddrives/Income_Dynamics_2022/FuzzyMatchesData/output_15/SHG_Details.csv")

# Only keep the state name Bihar
bihar_shg_details = df_shg_detail[df_shg_detail['State_Name'] == 'BIHAR'].drop_duplicates()

# Get State_Name, District_Name, Block_Name, Grampanchayat, Vilage_Name, SHGID_Numerical
SN = list()
DN = list()
BN = list()
PC = list()
VN = list()
GID = list()
for r in bihar_shg_details:
  SN = bihar_shg_details['State_Name'].values.tolist()
  DN = bihar_shg_details['District_Name'].values.tolist()
  BN = bihar_shg_details['Block_Name'].values.tolist()
  PC = bihar_shg_details['Grampanchayat'].values.tolist()
  VN = bihar_shg_details['Vilage_Name'].values.tolist()
  GID = bihar_shg_details['SHGID_Numerical'].values.tolist() # Not have to

# Read SHG_Members.csv
shg_members = 'SHG_Members.csv'
# df_shg_members = pd.read_csv(gdrive_path + shg_members, encoding = 'utf-16')
# df_shg_members = pd.read_csv("/content/gdrive/Shareddrives/Income_Dynamics_2022/FuzzyMatchesData/SHG_15/SHG_Members.csv")
# df_shg_members = pd.read_gsheet("/content/gdrive/Shareddrives/Income_Dynamics_2022/FuzzyMatchesData/SHG_Members.gsheet")
df_shg_members = pd.read_csv("/content/gdrive/Shareddrives/Income_Dynamics_2022/FuzzyMatchesData/output_15/SHG_Members_withid_15.csv")
df_shg_members.rename(columns = {'html_file_name':'HTML_File_Name', 'member_name':'Member_Name', 'fatherhusband_name':'Father.Husband_Name',	'gender':'Gender',	'social_category':'Social_Category', 'member_code': 'Member_Code'}, inplace = True)

# Only keep members that is in Bihar and combine the data with location information
bihar_shg_clean = pd.DataFrame()
idx = 0
for row in bihar_shg_details['HTML_File_Name']:
  sub_members = df_shg_members[df_shg_members['HTML_File_Name'] == row].drop_duplicates()
  sub_members['State_Name'] = SN[idx]
  sub_members['District_Name'] = DN[idx]
  sub_members['Block_Name'] = BN[idx]
  sub_members['Grampanchayat'] = PC[idx]
  sub_members['Vilage_Name'] = VN[idx]
  sub_members['SHGID_Numerical'] = GID[idx]
  bihar_shg_clean = pd.concat([bihar_shg_clean, sub_members])
  idx += 1
# bihar_shg_clean is the dataset that only contains members's informatoin who are in Bihar
bihar_shg_clean

,HTML_File_Name,sr_no,Member_Code,Member_Name,Father.Husband_Name,Gender,Social_Category,disability,sub_category_religion,aplbpl,pip_category,shg_internal_id,merge_ID,member_internal_subid,member_internal_uniqueid,State_Name,District_Name,Block_Name,Grampanchayat,Vilage_Name,SHGID_Numerical
0,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,12,9115091,MANEJA,OMRUL,Female,Other,No,Muslim,APL,POP,150073248,3,1,150073248001,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,524001001001
1,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,1,9115092,SABERA,NAJABUL,Female,Other,No,Muslim,APL,POP,150073248,3,2,150073248002,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,524001001001
2,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,10,9115248,SUGIYA,SAMFUL,Female,Other,No,Muslim,APL,POP,150073248,3,3,150073248003,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,524001001001
3,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,11,9115297,RUKHSANA KHATOON,CHHOTU,Female,Other,No,Muslim,APL,POP,150073248,3,4,150073248004,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,524001001001
4,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,2,9115298,RABILA,IDRISH,Female,Other,No,Muslim,APL,POP,150073248,3,5,150073248005,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,524001001001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,8,15834360,Resham Kumari,Munilal roy,Female,OBC,No,Hindu,NaN,POP,150073308,3,8,150073308008,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,524004010002
727,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,9,15834361,Guddi Kumari,Manoj Kumar Mandal,Female,OBC,No,Hindu,NaN,POP,150073308,3,9,150073308009,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,524004010002
728,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,10,15834362,Chandani Devi,Fuleshvar Mandal,Female,OBC,No,Hindu,NaN,POP,150073308,3,10,150073308010,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,524004010002
729,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,11,15834363,Savitri Devi,Bhola Mandal,Female,OBC,No,Hindu,NaN,POP,150073308,3,11,150073308011,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,524004010002


In [ ]:
# loop over all the files in the mergeTranslation and merge them into one: Hindi_Name
# # trans_path = "/content/gdrive/Shareddrives/Income_Dynamics_2022/MergedTranslations" #FIXME, uncomment for real
# trans_path = "/content/gdrive/Shareddrives/Income_Dynamics_2022/try"
# file_list = glob.glob(trans_path + "/*.xlsx")
# xfList = list()
# for x in file_list:
#   xf = pd.read_excel(x)
#   xfList.append(xf)
# Hindi_Name = pd.concat(xfList)

# # drop the unnessary columns
# Hindi_Name = Hindi_Name.drop(columns=["Unnamed: 0", "Unnamed: 4", "Unnamed: 5","eng_member_name", "eng_fatherhusband_name"] )

# # output: Hindi_Name
# Hindi_Name
Hindi_Name = pd.read_csv("/content/gdrive/Shareddrives/Income_Dynamics_2022/FuzzyMatchesData/Hindi_Name.csv", dtype={'uniqueid':'int'})
Hindi_Name =  Hindi_Name.drop(columns=["Unnamed: 0"])

Hindi_Name

,hindi_member_name,hindi_fatherhusband_name,uniqueid
0,नीलम देवी,राधे श्याम ठाकुर,200241007002
1,गायत्री देवी,उपेंद्र यादव,200241007003
2,आशा देवी,श्री नारायण यादव,200241007004
3,संभा देवी,ब्रह्मदेव यादव,200241007005
4,सांझा देवी,ओमप्रकाश यादव,200241007006
...,...,...,...
10820089,धर्मशीला देवी,प्रमोद राम,190040915008
10820090,उमरावती देवी,मंगानी रामी,190040915009
10820091,सुनीता देवी,मुन्ना रामी,190040915010
10820092,सुदामा देवी,लगान राम,190040915011


In [ ]:
#for Hindi_Name, we take the uniqueID (Hindi_Name) match with the Member_Code (shg data)
bihar_shg_clean = bihar_shg_clean.merge(Hindi_Name, left_on = 'member_internal_uniqueid', right_on = 'uniqueid', how='right').dropna(subset=['HTML_File_Name'])
bihar_shg_clean

,HTML_File_Name,sr_no,Member_Code,Member_Name,Father.Husband_Name,Gender,Social_Category,disability,sub_category_religion,aplbpl,pip_category,shg_internal_id,merge_ID,member_internal_subid,member_internal_uniqueid,State_Name,District_Name,Block_Name,Grampanchayat,Vilage_Name,SHGID_Numerical,hindi_member_name,hindi_fatherhusband_name,uniqueid
5810770,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,12.0,9115091.0,MANEJA,OMRUL,Female,Other,No,Muslim,APL,POP,150073248.0,3.0,1.0,1.500732e+11,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,5.240010e+11,मानेजा,ओमरुल,150073248001
5810771,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,1.0,9115092.0,SABERA,NAJABUL,Female,Other,No,Muslim,APL,POP,150073248.0,3.0,2.0,1.500732e+11,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,5.240010e+11,सबेरा,नजबूल,150073248002
5810772,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,10.0,9115248.0,SUGIYA,SAMFUL,Female,Other,No,Muslim,APL,POP,150073248.0,3.0,3.0,1.500732e+11,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,5.240010e+11,सुगिया,SAMFUL,150073248003
5810773,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,11.0,9115297.0,RUKHSANA KHATOON,CHHOTU,Female,Other,No,Muslim,APL,POP,150073248.0,3.0,4.0,1.500732e+11,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,5.240010e+11,रुखसाना खातून,छोटू,150073248004
5810774,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,2.0,9115298.0,RABILA,IDRISH,Female,Other,No,Muslim,APL,POP,150073248.0,3.0,5.0,1.500732e+11,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,5.240010e+11,रबीला,इदरिश,150073248005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5811496,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,8.0,15834360.0,Resham Kumari,Munilal roy,Female,OBC,No,Hindu,NaN,POP,150073308.0,3.0,8.0,1.500733e+11,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,5.240040e+11,रेशम कुमारी,मुनीलाल रॉय,150073308008
5811497,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,9.0,15834361.0,Guddi Kumari,Manoj Kumar Mandal,Female,OBC,No,Hindu,NaN,POP,150073308.0,3.0,9.0,1.500733e+11,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,5.240040e+11,गुड्डी कुमारी,मनोज कुमार मंडल,150073308009
5811498,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,10.0,15834362.0,Chandani Devi,Fuleshvar Mandal,Female,OBC,No,Hindu,NaN,POP,150073308.0,3.0,10.0,1.500733e+11,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,5.240040e+11,चांदनी देवी,फुलेश्वर मंडल,150073308010
5811499,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,11.0,15834363.0,Savitri Devi,Bhola Mandal,Female,OBC,No,Hindu,NaN,POP,150073308.0,3.0,11.0,1.500733e+11,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,5.240040e+11,सावित्री देवी,भोला मंडल,150073308011


In [ ]:
# Jobcard + familyDetails by jobcardno = Job_Info
# File locations
Job_file_path = "/content/gdrive/Shareddrives/Income_Dynamics_2022/Job Card Example data/raid/users/azshenoy/Bihar/parse/final_output/districtwise/15/JobCard.dta"
Family_file_path = "/content/gdrive/Shareddrives/Income_Dynamics_2022/Job Card Example data/raid/users/azshenoy/Bihar/parse/final_output/districtwise/15/FamilyDet.dta"

# Load into Pandas dataframe
dfJob = pd.read_stata(Job_file_path)
dfFamily = pd.read_stata(Family_file_path)

# Merge the dataframes
dfjobcard_merged = pd.merge(dfJob, dfFamily, on='jobcardno', how='inner', suffixes=('', '_y'))

# Drop the duplicate columns
dfjobcard_merged.drop(dfjobcard_merged.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [ ]:
# pd.set_option('display.max_columns', 10)
# dfjobcard_merged.head()

# dfjobcard_merged[dfjobcard_merged["jobcardno"].duplicated()]
# dfjobcard_merged[dfjobcard_merged["jobcardno"]=="BH-24-001-001-00829500/1000"]

dfjobcard_merged[(dfjobcard_merged["name"]!=dfjobcard_merged["name"])]
dfjobcard_merged

,jobcardno,castegroup,panchayat,block,district,isbpl,bplno,familyid,headname,malerel,regdate,village,epicno,src,did,si,name,gender,age,bank_or_post


,jobcardno,castegroup,panchayat,block,district,isbpl,bplno,familyid,headname,malerel,regdate,village,epicno,src,did,si,name,gender,age,bank_or_post
0,BH-24-004-021-00864900/1441,OTH,SIKORNA,KADWA,KATIHAR(BIHAR),NO,,1441,तेतली देवी,लुखेय चौधरी,4/22/2010,SIKORNA,,1. output/output_index20190617_1,15.0,1,तेतली देवी,Male,36,State Bank of India
1,BH-24-012-003-00947200/1083,OTH,BHAMRAILI,DANDKHORA,KATIHAR(BIHAR),NO,,1083,घनश्‍याम केवट,शंकर केवट,10/31/2007,RAGHAILI,,1. output/output_index20190617_1,15.0,2,छोटी देवी,Female,28,
2,BH-24-012-003-00947200/1083,OTH,BHAMRAILI,DANDKHORA,KATIHAR(BIHAR),NO,,1083,घनश्‍याम केवट,शंकर केवट,10/31/2007,RAGHAILI,,1. output/output_index20190617_1,15.0,1,घनश्‍याम केवट,Male,35,
3,BH-24-004-019-00867800/730,OTH,BHOU NAGAR,KADWA,KATIHAR(BIHAR),NO,,730,मो0 मुशताक आलम,स्‍व0 फरीद,5/11/2006,BHAOU NAGAR,,1. output/output_index20190617_1,15.0,1,मो0 मुशताक आलम,Male,56,
4,BH-24-002-003-00839700/2801,ST,MUSAPUR,KORHA,KATIHAR(BIHAR),YES,76630,2801,HONMOY DEVI,GARAJIT SOREN,4/1/2015,MUSAPUR,,1. output/output_index20190617_1,15.0,1,HONMOY DEVI,Female,23,United Bank Of India
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851502,BH-24-004-022-00863800/2690,OTH,KADWA,KADWA,KATIHAR(BIHAR),NO,,2690,बेदया देवी,अक्षय विश्‍वास,8/8/2009,KADWA,,18. output/output_index20190617_25,15.0,3,राम विलास विश्‍वास,Male,22,
851503,BH-24-004-005-00858300/163-A,SC,PERBHELI,KADWA,KATIHAR(BIHAR),NO,,163-A,अर्जुन विश्‍वास,श्री लाल विश्‍वास,6/24/2006,KARMELI,,18. output/output_index20190617_25,15.0,1,अर्जुन विश्‍वास,Male,25,State Bank of India
851504,BH-24-004-005-00858300/163-A,SC,PERBHELI,KADWA,KATIHAR(BIHAR),NO,,163-A,अर्जुन विश्‍वास,श्री लाल विश्‍वास,6/24/2006,KARMELI,,18. output/output_index20190617_25,15.0,2,सुमितरा देवी,Female,45,
851505,BH-24-004-005-00858300/163-A,SC,PERBHELI,KADWA,KATIHAR(BIHAR),NO,,163-A,अर्जुन विश्‍वास,श्री लाल विश्‍वास,6/24/2006,KARMELI,,18. output/output_index20190617_25,15.0,3,प्रिति देवी,Female,23,State Bank of India


In [ ]:
# Create numerical id for location in bihar_shg_clean
bihar_shg_clean.insert(loc=0, column='SHG_Location_ID', value=bihar_shg_clean.set_index(['State_Name','District_Name', 'Block_Name', 'Grampanchayat', 'Vilage_Name']).index.factorize()[0]+1)

# get the combination of locations with their location_id in SHG dataset, no duplicates
location_shg = bihar_shg_clean[['State_Name','District_Name', 'Block_Name', 'Grampanchayat', 'Vilage_Name', 'SHG_Location_ID']].drop_duplicates()

In [ ]:
# Makes separate column for state and fixes district string value
dfjobcard_merged['districts'] = dfjobcard_merged.apply(lambda x: x['district'].split('(',1)[0].upper(), axis=1)
dfjobcard_merged['state'] = dfjobcard_merged.apply(lambda x: x['district'].split('(',1)[1].upper(), axis=1)
dfjobcard_merged['state'] = dfjobcard_merged['state'].str.replace(')','')

# assign unique numerical id for each combination of the location in jobCard datasets
dfjobcard_merged.insert(loc=0, column='location_id', value=dfjobcard_merged.set_index(['districts', 'block', 'panchayat', 'village', 'state']).index.factorize()[0]+1)

# get the combination of locations with their location_id in jobCard dataset, no duplicates
location_jobcard = dfjobcard_merged[['districts', 'block', 'panchayat', 'village', 'state', 'location_id']].drop_duplicates()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


In [ ]:
location_shg

,State_Name,District_Name,Block_Name,Grampanchayat,Vilage_Name,SHG_Location_ID
5810770,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,1
5811034,BIHAR,KATIHAR,AMDABAD,BAIRIYA,JAMRA,2
5811048,BIHAR,KATIHAR,AZAMNAGAR,JALKI,JALKI,3
5811072,BIHAR,KATIHAR,AZAMNAGAR,JALKI,PARLA,4
5811112,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,JAKAR,5
5811124,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,MARIA,6
5811183,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,PALSA,7
5811193,BIHAR,KATIHAR,BARARI,BARARI,BARARI,8
5811359,BIHAR,KATIHAR,BARARI,BARETA,BARETHA,9
5811403,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,10


In [ ]:
location_jobcard

,districts,block,panchayat,village,state,location_id
0,KATIHAR,KADWA,SIKORNA,SIKORNA,BIHAR,1
1,KATIHAR,DANDKHORA,BHAMRAILI,RAGHAILI,BIHAR,2
3,KATIHAR,KADWA,BHOU NAGAR,BHAOU NAGAR,BIHAR,3
4,KATIHAR,KORHA,MUSAPUR,MUSAPUR,BIHAR,4
6,KATIHAR,BARARI,BISHNPUR,BISHANPUR,BIHAR,5
...,...,...,...,...,...,...
707457,KATIHAR,KADWA,SHEKHPURA,MADHYAM TOLA,BIHAR,1317
748374,KATIHAR,KADWA,PELAGARH,DIYARI,BIHAR,1318
773442,KATIHAR,BALRAMPUR,LUTTIPUR,NISHAT MAL,BIHAR,1319
786743,KATIHAR,BARARI,RAUNIA,MAKHNAHA MILIK,BIHAR,1320


In [ ]:
# Fuzy matching block on state Bihar
ncores = 1
percentM = 0.75 # set the percent for matching to 75%
def makelink(dftuple):
  SHG_location = dftuple[0]
  JF_location = dftuple[1]

  indexer  = recordlinkage.Index()
  indexer.block(left_on=['State_Name'], right_on=['state']) # left on SHG, right on jobcard
  candidates = indexer.index(SHG_location, JF_location)

  if len(candidates) == 0:
    return pd.DataFrame()

  # campare the location information in both SHG and jobCard datasets
  compare = recordlinkage.Compare()
  compare.string('District_Name', 'districts', label='Districts')
  compare.string('Block_Name', 'block', label='Block')
  compare.string('Grampanchayat', 'panchayat', label='Panchayat')
  compare.string('Vilage_Name', 'village', label='Village')

  # keep result that is greater than match percentage
  match_table = compare.compute(candidates, SHG_location, JF_location)
  potential_matches = match_table[ match_table['Districts'] >= percentM ].reset_index()
  potential_matches = match_table[ match_table['Block'] >= percentM ].reset_index()
  potential_matches = match_table[ match_table['Panchayat'] >= percentM ].reset_index()
  potential_matches = match_table[ match_table['Village'] >= percentM ].reset_index()

  tmapleft = SHG_location.loc[ potential_matches['level_0'] ].reset_index().rename(columns={'index':'index_SHG_location'})
  tmapright = JF_location.loc[ potential_matches['level_1']].reset_index().rename(columns={'index':'index_JF_location'})

  # Merge results
  mergeddata = tmapleft.join(tmapright).join(potential_matches)

  # filter out rows that don't have matching percentage greater than percentM
  mergeddata = mergeddata.loc[mergeddata['Districts'] >= percentM].loc[mergeddata['Block'] >= percentM].loc[mergeddata['Panchayat'] >= percentM].loc[mergeddata['Village'] >= percentM]

  return mergeddata

In [ ]:
# Split dataframes into lists based on first letter of district name
# call the funcction without sorting
SHGBlockList = [location_shg.copy()]
JFBlockList = [location_jobcard.copy()]

# call first fuzzy matching function
dflist = list(zip(SHGBlockList, JFBlockList))
with Pool(ncores) as p:
  donelist = p.map(makelink, dflist)
  
# concat donelist 
allmatches = pd.concat(donelist)

In [ ]:
# The final output for fuzzy matches block on locations 
allmatches

,index_SHG_location,State_Name,District_Name,Block_Name,Grampanchayat,Vilage_Name,SHG_Location_ID,index_JF_location,districts,block,panchayat,village,state,location_id,level_0,level_1,Districts,Block,Panchayat,Village
0,5810770,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,1,147,KATIHAR,AMDABAD,BAIDA,BAIDA,BIHAR,93,5810770,147,1.0,1.0,1.000000,1.000000
5,5811034,BIHAR,KATIHAR,AMDABAD,BAIRIYA,JAMRA,2,2201,KATIHAR,AMDABAD,BAIRIA,JAMRA,BIHAR,614,5811034,2201,1.0,1.0,0.857143,1.000000
6,5811048,BIHAR,KATIHAR,AZAMNAGAR,JALKI,JALKI,3,1926,KATIHAR,AZAMNAGAR,JALKI,JALKI,BIHAR,575,5811048,1926,1.0,1.0,1.000000,1.000000
7,5811072,BIHAR,KATIHAR,AZAMNAGAR,JALKI,PARLA,4,996,KATIHAR,AZAMNAGAR,JALKI,PORLA,BIHAR,393,5811072,996,1.0,1.0,1.000000,0.800000
9,5811112,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,JAKAR,5,156,KATIHAR,AZAMNAGAR,JOKAR,JOKAR,BIHAR,100,5811112,156,1.0,1.0,1.000000,0.800000
14,5811124,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,MARIA,6,47898,KATIHAR,AZAMNAGAR,JOKAR,MARIYA,BIHAR,1187,5811124,47898,1.0,1.0,1.000000,0.833333
15,5811183,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,PALSA,7,3408,KATIHAR,AZAMNAGAR,JOKAR,PALSA,BIHAR,733,5811183,3408,1.0,1.0,1.000000,1.000000
16,5811193,BIHAR,KATIHAR,BARARI,BARARI,BARARI,8,13,KATIHAR,BARARI,BARARI,BARARI,BIHAR,10,5811193,13,1.0,1.0,1.000000,1.000000
19,5811359,BIHAR,KATIHAR,BARARI,BARETA,BARETHA,9,1019,KATIHAR,BARARI,BARETA,BARETA,BIHAR,398,5811359,1019,1.0,1.0,1.000000,0.857143
20,5811403,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,10,67,KATIHAR,BARARI,KABAR,KABAR,BIHAR,45,5811403,67,1.0,1.0,0.800000,1.000000


Fuzzy match names:

In [ ]:
# Prepare the dataframes for merge 
shg_merge = allmatches[['State_Name', 'District_Name', 'Block_Name', 'Grampanchayat', 'Vilage_Name', 'location_id']].copy()
shg_merge = shg_merge.rename(columns={'location_id': 'SHG_Zipcode_location_id'})
jobcard_merge = allmatches[['state', 'districts', 'block', 'panchayat', 'village', 'location_id']].copy()
jobcard_merge = jobcard_merge.rename(columns={'location_id': 'Zipcode_location_id'})

# # Merge the Location_ID back to the original dataset
# shg_second_fuzzy = bihar_shg_clean.merge(shg_merge, left_on=['State_Name', 'District_Name', 'Block_Name', 'Grampanchayat', 'Vilage_Name'], right_on=['State_Name', 'District_Name', 'Block_Name', 'Grampanchayat', 'Vilage_Name'])
# jobcard_second_fuzzy = dfjobcard_merged.merge(jobcard_merge, left_on=['state', 'districts', 'block', 'panchayat', 'village'], right_on=['state', 'districts', 'block', 'panchayat', 'village'])

# # Only fuzzy match the non-empty rows
# jobcard_second_fuzzy.malerel.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
# jobcard_second_fuzzy.headname.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
# jobcard_second_fuzzy = jobcard_second_fuzzy[jobcard_second_fuzzy['malerel'].notna()]
# jobcard_second_fuzzy = jobcard_second_fuzzy[jobcard_second_fuzzy['headname'].notna()]
# jobcard_second_fuzzy = jobcard_second_fuzzy.drop('location_id', 1) 

In [ ]:
shg_merge

,State_Name,District_Name,Block_Name,Grampanchayat,Vilage_Name,SHG_Zipcode_location_id
0,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,93
5,BIHAR,KATIHAR,AMDABAD,BAIRIYA,JAMRA,614
6,BIHAR,KATIHAR,AZAMNAGAR,JALKI,JALKI,575
7,BIHAR,KATIHAR,AZAMNAGAR,JALKI,PARLA,393
9,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,JAKAR,100
14,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,MARIA,1187
15,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,PALSA,733
16,BIHAR,KATIHAR,BARARI,BARARI,BARARI,10
19,BIHAR,KATIHAR,BARARI,BARETA,BARETHA,398
20,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,45


In [ ]:
jobcard_merge

,state,districts,block,panchayat,village,Zipcode_location_id
0,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,93
5,BIHAR,KATIHAR,AMDABAD,BAIRIA,JAMRA,614
6,BIHAR,KATIHAR,AZAMNAGAR,JALKI,JALKI,575
7,BIHAR,KATIHAR,AZAMNAGAR,JALKI,PORLA,393
9,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,JOKAR,100
14,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,MARIYA,1187
15,BIHAR,KATIHAR,AZAMNAGAR,JOKAR,PALSA,733
16,BIHAR,KATIHAR,BARARI,BARARI,BARARI,10
19,BIHAR,KATIHAR,BARARI,BARETA,BARETA,398
20,BIHAR,KATIHAR,BARARI,KABAR,KABAR,45


In [ ]:
# Make sure the headname are all male
# If the headname is male, nothing change (done)
# If the headname is female or empty, the malerel is not empty, then change the headname to the malerel (done)
# If headname is female or empty, malerel is empty
#     If there is a unique male in the family, then assign him as the headname (done)
#     If there is no male, ???
#     If there are multiple males, ???
# didn't see last two cases in the dataset we have

# make copies
df = dfjobcard_merged.copy()
df2 = dfjobcard_merged.copy()

# If the headname is female
df = df[(df['headname'] == df['name']) & (df['gender'] == 'Female')]
# change the headname to the malerel
df2 = df2[df2['headname'].isin(df['headname'])]
df2['headname'] = df2['malerel']
# add the changed dataframe back to the original dataframe
dfjobcard_merged.update(df2)
dfjobcard_merged.drop(['malerel'], axis=1, inplace=True) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Second Stage Fuzzy matching block on numerical identifiers
ncores = 1
MatchPercent = 0.75
def makelink2(dftuple) :
    shg_second_fuzzy = dftuple[0]
    jobcard_second_fuzzy = dftuple[1]

    indexer  = recordlinkage.Index() # The Python Record Linkage Toolkit is a library to link records in or between data sources, use index
    indexer.block(left_on=['SHG_Zipcode_location_id', 'Social_Category', 'Gender'], right_on=['Zipcode_location_id', 'castegroup', 'gender']) # left is allmatches, right is dfjobcard_merged
    candidates = indexer.index(shg_second_fuzzy, jobcard_second_fuzzy)

    # FIXME: can also block on gender and caste group

    if len(candidates) == 0:
        return pd.DataFrame() # return an empty dataframe
    
    # Compare names
    compare = recordlinkage.Compare()
    compare.string('Member_Name', 'name', label='matched_name') 
    compare.string('Father.Husband_Name', 'headname', label='matched_father_husband_name')
    
    # only keep above 75%
    match_table = compare.compute(candidates, shg_second_fuzzy, jobcard_second_fuzzy)
    potential_matches = match_table[ match_table['matched_name'] >= MatchPercent ].reset_index()
    potential_matches = match_table[ match_table['matched_father_husband_name'] >= MatchPercent ].reset_index()

    tmapleft = shg_second_fuzzy.loc[ potential_matches['level_0'] ].reset_index().rename(columns={'index':'index_SHG_name'})
    tmapright = jobcard_second_fuzzy.loc[ potential_matches['level_1']].reset_index().rename(columns={'index':'index_JF_name'})

    SHG_jobcard_final_mergeddata = tmapleft.join(tmapright).join(potential_matches)

    # filter out rows that are less than match percentage
    SHG_jobcard_final_mergeddata = SHG_jobcard_final_mergeddata.loc[SHG_jobcard_final_mergeddata['matched_name'] >= MatchPercent].loc[SHG_jobcard_final_mergeddata['matched_father_husband_name'] >= MatchPercent]

    return SHG_jobcard_final_mergeddata

In [ ]:
dfshg_second_fuzzy = bihar_shg_clean.merge(shg_merge, left_on=['State_Name', 'District_Name', 'Block_Name', 'Grampanchayat', 'Vilage_Name'], right_on=['State_Name', 'District_Name', 'Block_Name', 'Grampanchayat', 'Vilage_Name'])
dfjobcard_second_fuzzy = dfjobcard_merged.merge(jobcard_merge, left_on=['state', 'districts', 'block', 'panchayat', 'village'], right_on=['state', 'districts', 'block', 'panchayat', 'village'])

In [ ]:
dfshg_second_fuzzy

,SHG_Location_ID,HTML_File_Name,sr_no,Member_Code,Member_Name,Father.Husband_Name,Gender,Social_Category,disability,sub_category_religion,aplbpl,pip_category,shg_internal_id,merge_ID,member_internal_subid,member_internal_uniqueid,State_Name,District_Name,Block_Name,Grampanchayat,Vilage_Name,SHGID_Numerical,hindi_member_name,hindi_fatherhusband_name,uniqueid,SHG_Zipcode_location_id
0,1,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,12.0,9115091.0,MANEJA,OMRUL,Female,Other,No,Muslim,APL,POP,150073248.0,3.0,1.0,1.500732e+11,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,5.240010e+11,मानेजा,ओमरुल,150073248001,93
1,1,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,1.0,9115092.0,SABERA,NAJABUL,Female,Other,No,Muslim,APL,POP,150073248.0,3.0,2.0,1.500732e+11,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,5.240010e+11,सबेरा,नजबूल,150073248002,93
2,1,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,10.0,9115248.0,SUGIYA,SAMFUL,Female,Other,No,Muslim,APL,POP,150073248.0,3.0,3.0,1.500732e+11,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,5.240010e+11,सुगिया,SAMFUL,150073248003,93
3,1,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,11.0,9115297.0,RUKHSANA KHATOON,CHHOTU,Female,Other,No,Muslim,APL,POP,150073248.0,3.0,4.0,1.500732e+11,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,5.240010e+11,रुखसाना खातून,छोटू,150073248004,93
4,1,BIHAR_KATIHAR_AMDABAD_BAIDA_BAIDA_AFSANA_1.htm...,2.0,9115298.0,RABILA,IDRISH,Female,Other,No,Muslim,APL,POP,150073248.0,3.0,5.0,1.500732e+11,BIHAR,KATIHAR,AMDABAD,BAIDA,BAIDA,5.240010e+11,रबीला,इदरिश,150073248005,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,10,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,8.0,15834360.0,Resham Kumari,Munilal roy,Female,OBC,No,Hindu,NaN,POP,150073308.0,3.0,8.0,1.500733e+11,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,5.240040e+11,रेशम कुमारी,मुनीलाल रॉय,150073308008,45
727,10,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,9.0,15834361.0,Guddi Kumari,Manoj Kumar Mandal,Female,OBC,No,Hindu,NaN,POP,150073308.0,3.0,9.0,1.500733e+11,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,5.240040e+11,गुड्डी कुमारी,मनोज कुमार मंडल,150073308009,45
728,10,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,10.0,15834362.0,Chandani Devi,Fuleshvar Mandal,Female,OBC,No,Hindu,NaN,POP,150073308.0,3.0,10.0,1.500733e+11,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,5.240040e+11,चांदनी देवी,फुलेश्वर मंडल,150073308010,45
729,10,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASTHA_11.html...,11.0,15834363.0,Savitri Devi,Bhola Mandal,Female,OBC,No,Hindu,NaN,POP,150073308.0,3.0,11.0,1.500733e+11,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,5.240040e+11,सावित्री देवी,भोला मंडल,150073308011,45


In [ ]:
dfjobcard_second_fuzzy

,location_id,jobcardno,castegroup,panchayat,block,district,isbpl,bplno,familyid,headname,regdate,village,epicno,src,did,si,name,gender,age,bank_or_post,districts,state,Zipcode_location_id
0,10.0,BH-24-009-002-00965400/1703,OTH,BARARI,BARARI,KATIHAR(BIHAR),NO,,1703,रामनाथ चौरसिया,6/28/2009,BARARI,,1. output/output_index20190617_1,15.0,1.0,रामनाथ चौरसिया,Male,33.0,,KATIHAR,BIHAR,10
1,10.0,BH-24-009-002-00965400/1703,OTH,BARARI,BARARI,KATIHAR(BIHAR),NO,,1703,रामनाथ चौरसिया,6/28/2009,BARARI,,1. output/output_index20190617_1,15.0,2.0,नीलम देवी,Female,28.0,,KATIHAR,BIHAR,10
2,10.0,BH-24-009-002-00965400/131-A,ST,BARARI,BARARI,KATIHAR(BIHAR),NO,,131,स्‍व0 सुन्‍दर सोरेन,2/1/2007,BARARI,,1. output/output_index20190617_1,15.0,1.0,मसो0 बडकी देवी,Female,40.0,,KATIHAR,BIHAR,10
3,10.0,BH-24-009-002-00965400/75-A,SC,BARARI,BARARI,KATIHAR(BIHAR),NO,,75,संतोष ऋषि,2/1/2007,BARARI,,1. output/output_index20190617_1,15.0,1.0,संतोष\t ऋषि,Male,24.0,,KATIHAR,BIHAR,10
4,10.0,BH-24-009-002-00965400/1451,OTH,BARARI,BARARI,KATIHAR(BIHAR),NO,,1451,विनोद चौहान,5/28/2009,BARARI,,1. output/output_index20190617_1,15.0,1.0,विनोद चौहान,Male,30.0,,KATIHAR,BIHAR,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14402,1187.0,BH-24-007-008-00930300/702,OTH,JOKAR,AZAMNAGAR,KATIHAR(BIHAR),NO,,702,उसमान अंसार,3/6/2006,MARIYA,,17. output/output_index20190617_24,15.0,1.0,मसो0 डोमनी,Female,32.0,,KATIHAR,BIHAR,1187
14403,1187.0,BH-24-007-008-00930300/2493,OTH,JOKAR,AZAMNAGAR,KATIHAR(BIHAR),NO,,2493,LALIT CHAUDHARI,3/15/2017,MARIYA,,17. output/output_index20190617_24,15.0,1.0,CHANCHALA DEVI,Female,40.0,,KATIHAR,BIHAR,1187
14404,1187.0,BH-24-007-008-00930300/2491,OTH,JOKAR,AZAMNAGAR,KATIHAR(BIHAR),NO,,2491,DULAL CHAUDHARI,3/15/2017,MARIYA,,17. output/output_index20190617_24,15.0,1.0,BHOLIYA DEVI,Female,40.0,,KATIHAR,BIHAR,1187
14405,1187.0,BH-24-007-008-00930300/2495,OTH,JOKAR,AZAMNAGAR,KATIHAR(BIHAR),NO,,2495,KAILASH PARIHAR,3/15/2017,MARIYA,,18. output/output_index20190617_25,15.0,1.0,ANITA DEVI,Female,40.0,State Bank of India,KATIHAR,BIHAR,1187


In [ ]:
# call the fuzzy match to match the English names
second_SHGBlockList = [dfshg_second_fuzzy.copy()]
second_JFBlockList = [dfjobcard_second_fuzzy.copy()]

#dflist = list(zip(SHGBlockList, JFBlockList))
dflist_second = list(zip(second_SHGBlockList, second_JFBlockList))
with Pool(ncores) as p:
  second_donelist = p.map(makelink2, dflist_second)
finalmatches = pd.concat(second_donelist)

In [ ]:
# result of matching English names
finalmatches

,index_SHG_name,SHG_Location_ID,HTML_File_Name,sr_no,Member_Code,Member_Name,Father.Husband_Name,Gender,Social_Category,disability,sub_category_religion,aplbpl,pip_category,shg_internal_id,merge_ID,member_internal_subid,member_internal_uniqueid,State_Name,District_Name,Block_Name,Grampanchayat,Vilage_Name,SHGID_Numerical,hindi_member_name,hindi_fatherhusband_name,uniqueid,SHG_Zipcode_location_id,index_JF_name,location_id,jobcardno,castegroup,panchayat,block,district,isbpl,bplno,familyid,headname,regdate,village,epicno,src,did,si,name,gender,age,bank_or_post,districts,state,Zipcode_location_id,level_0,level_1,matched_name,matched_father_husband_name
0,472,8,BIHAR_KATIHAR_BARARI_BARARI_BARARI_KALI_33.htm...,4.0,9136545.0,ANITA DEVI,NIGAM KUMAR RAM,Female,SC,No,Hindu,APL,POP,150073287.0,3.0,4.0,1.500733e+11,BIHAR,KATIHAR,BARARI,BARARI,BARARI,5.240040e+11,अनीता देवी,निगम कुमार राम,150073287004,10,2232,10.0,BH-24-009-002-00965400/2287,SC,BARARI,BARARI,KATIHAR(BIHAR),NO,,2287,NIGAM KR RAM,4/1/2014,BARARI,,10. output/output_index20190617_18,15.0,1.0,ANITA DEVI,Female,30.0,,KATIHAR,BIHAR,10,472,2232,1.000000,0.800000
1,475,8,BIHAR_KATIHAR_BARARI_BARARI_BARARI_KALI_33.htm...,6.0,9137069.0,SULEKHA DEVI,ARUN RAM,Female,SC,No,Hindu,APL,POP,150073287.0,3.0,7.0,1.500733e+11,BIHAR,KATIHAR,BARARI,BARARI,BARARI,5.240040e+11,सुलेखा देवी,अरुण राम,150073287007,10,1197,10.0,BH-24-009-002-00965400/2764,SC,BARARI,BARARI,KATIHAR(BIHAR),NO,,2764,ARUN RAM,6/1/2017,BARARI,,25. output/output_index20190617_9,15.0,1.0,SULEKHA DEVI,Female,59.0,,KATIHAR,BIHAR,10,475,1197,1.000000,1.000000
4,568,8,BIHAR_KATIHAR_BARARI_BARARI_BARARI_SAAI_94.htm...,6.0,15836223.0,Sekha Devi,Manoj Rishi,Female,SC,No,Hindu,NaN,POP,150073295.0,3.0,6.0,1.500733e+11,BIHAR,KATIHAR,BARARI,BARARI,BARARI,5.240040e+11,सेखा देवी,मनोज ऋषि,150073295006,10,2216,10.0,BH-24-009-002-00965400/1870,SC,BARARI,BARARI,KATIHAR(BIHAR),NO,,1870,manoj rishi,4/1/2014,BARARI,,10. output/output_index20190617_18,15.0,1.0,sekha devi,Female,35.0,Allahabad Bank,KATIHAR,BIHAR,10,568,2216,0.800000,0.818182
5,570,8,BIHAR_KATIHAR_BARARI_BARARI_BARARI_SAAI_94.htm...,8.0,15836225.0,Mina Devi,Mukesh rishi,Female,SC,No,Hindu,NaN,POP,150073295.0,3.0,8.0,1.500733e+11,BIHAR,KATIHAR,BARARI,BARARI,BARARI,5.240040e+11,मीना देवी,मुकेश ऋषि,150073295008,10,1621,10.0,BH-24-009-002-00965400/1882,SC,BARARI,BARARI,KATIHAR(BIHAR),NO,,1882,mukesh rishi,4/1/2014,BARARI,,5. output/output_index20190617_13,15.0,1.0,mina devi,Female,35.0,Allahabad Bank,KATIHAR,BIHAR,10,570,1621,0.777778,0.916667


In [ ]:
# We need to concider two cases when fuzzy match the names: English names and Hindi names
# Method: run the fuzzy match once, then switch English names columns and Hindi names columns and run the fuzzy match again
dfshg_second_fuzzy_copy = dfshg_second_fuzzy.copy(deep=True)
dfshg_second_fuzzy_copy = dfshg_second_fuzzy_copy.rename(columns={"Member_Name": "Ori_Names", "Father.Husband_Name": "Ori_Father.Husband_Name", "hindi_member_name": "Member_Name", "hindi_fatherhusband_name": "Father.Husband_Name"})

# Prepare new list. No change on second_JFBlockList 
second_SHGBlockList = [dfshg_second_fuzzy_copy.copy()]
# Call the fuzzy match again
dflist_second = list(zip(second_SHGBlockList, second_JFBlockList))
with Pool(ncores) as p:
  second_donelist = p.map(makelink2, dflist_second)
finalmatches_Hindi = pd.concat(second_donelist)

In [ ]:
# result of matching Hindi names
finalmatches_Hindi

,index_SHG_name,SHG_Location_ID,HTML_File_Name,sr_no,Member_Code,Ori_Names,Ori_Father.Husband_Name,Gender,Social_Category,disability,sub_category_religion,aplbpl,pip_category,shg_internal_id,merge_ID,member_internal_subid,member_internal_uniqueid,State_Name,District_Name,Block_Name,Grampanchayat,Vilage_Name,SHGID_Numerical,Member_Name,Father.Husband_Name,uniqueid,SHG_Zipcode_location_id,index_JF_name,location_id,jobcardno,castegroup,panchayat,block,district,isbpl,bplno,familyid,headname,regdate,village,epicno,src,did,si,name,gender,age,bank_or_post,districts,state,Zipcode_location_id,level_0,level_1,matched_name,matched_father_husband_name
5,581,8,BIHAR_KATIHAR_BARARI_BARARI_BARARI_SAAI_95.htm...,7.0,15813325.0,Sona devi,Hemlal Murmu,Female,ST,No,Hindu,NaN,POP,150073296.0,3.0,7.0,1.500733e+11,BIHAR,KATIHAR,BARARI,BARARI,BARARI,5.240040e+11,सोना देवी,हेमलाल मुर्मू,150073296007,10,2510,10.0,BH-24-009-002-00965400/1546,ST,BARARI,BARARI,KATIHAR(BIHAR),NO,,1546,हेमलाल मुर्मू,6/28/2009,BARARI,,13. output/output_index20190617_20,15.0,2.0,सोना देवी,Female,20.0,,KATIHAR,BIHAR,10,581,2510,1.000000,1.000000
8,582,8,BIHAR_KATIHAR_BARARI_BARARI_BARARI_SAAI_95.htm...,8.0,15813365.0,lalita Devi,Bitta Soren,Female,ST,No,Hindu,NaN,POP,150073296.0,3.0,8.0,1.500733e+11,BIHAR,KATIHAR,BARARI,BARARI,BARARI,5.240040e+11,ललिता देवी,बिट्टा सोरेन,150073296008,10,2977,10.0,BH-24-009-002-00965400/1154,ST,BARARI,BARARI,KATIHAR(BIHAR),NO,,1154,बिट्टा सोरेन,6/28/2008,BARARI,,17. output/output_index20190617_24,15.0,2.0,ललीता देवी,Female,25.0,,KATIHAR,BIHAR,10,582,2977,0.900000,1.000000
9,587,8,BIHAR_KATIHAR_BARARI_BARARI_BARARI_SAAI_95.htm...,13.0,15825963.0,Sanjhli Devi,Siral Soren,Female,ST,No,Hindu,NaN,POP,150073296.0,3.0,13.0,1.500733e+11,BIHAR,KATIHAR,BARARI,BARARI,BARARI,5.240040e+11,सांझली देवी,सिराल सोरेन,150073296013,10,2395,10.0,BH-24-009-002-00965400/1151,ST,BARARI,BARARI,KATIHAR(BIHAR),NO,,1151,सुराय सोरेन,6/28/2008,BARARI,,11. output/output_index20190617_19,15.0,2.0,संझली देवी,Female,35.0,,KATIHAR,BIHAR,10,587,2395,0.909091,0.818182
15,713,10,BIHAR_KATIHAR_BARARI_KAWAR_KABAR_ASHA_9.html.html,7.0,16493671.0,pavo Devi,Ramdev roy,Female,ST,No,Hindu,NaN,POP,150073307.0,3.0,7.0,1.500733e+11,BIHAR,KATIHAR,BARARI,KAWAR,KABAR,5.240040e+11,पावो देवी,रामदेव रॉय,150073307007,45,3981,45.0,BH-24-009-020-00958200/84-A,ST,KABAR,BARARI,KATIHAR(BIHAR),NO,,84,रामदेव उरॉंव,4/1/2006,KABAR,,23. output/output_index20190617_7,15.0,2.0,पावो देवी,Female,31.0,,KATIHAR,BIHAR,45,713,3981,1.000000,0.750000


In [ ]:
# result of matching English names (only show names)
finalmatches.loc[:,["Member_Name", "name", "matched_name", "Father.Husband_Name", "headname", "matched_father_husband_name", "location_id"]]

,Member_Name,name,matched_name,Father.Husband_Name,headname,matched_father_husband_name,location_id
0,ANITA DEVI,ANITA DEVI,1.000000,NIGAM KUMAR RAM,NIGAM KR RAM,0.800000,10.0
1,SULEKHA DEVI,SULEKHA DEVI,1.000000,ARUN RAM,ARUN RAM,1.000000,10.0
4,Sekha Devi,sekha devi,0.800000,Manoj Rishi,manoj rishi,0.818182,10.0
5,Mina Devi,mina devi,0.777778,Mukesh rishi,mukesh rishi,0.916667,10.0


In [ ]:
# result of matching Hindi names (only show names)
finalmatches_Hindi.loc[:,["Member_Name", "name", "matched_name", "Father.Husband_Name", "headname", "matched_father_husband_name", "location_id"]]

,Member_Name,name,matched_name,Father.Husband_Name,headname,matched_father_husband_name,location_id
5,सोना देवी,सोना देवी,1.000000,हेमलाल मुर्मू,हेमलाल मुर्मू,1.000000,10.0
8,ललिता देवी,ललीता देवी,0.900000,बिट्टा सोरेन,बिट्टा सोरेन,1.000000,10.0
9,सांझली देवी,संझली देवी,0.909091,सिराल सोरेन,सुराय सोरेन,0.818182,10.0
15,पावो देवी,पावो देवी,1.000000,रामदेव रॉय,रामदेव उरॉंव,0.750000,45.0


In [ ]:
# Statistic
overallMatchSHG = (len(finalmatches) + len(finalmatches_Hindi)) / len(location_shg) * 100
overallMatchJC = (len(finalmatches) + len(finalmatches_Hindi)) / len(location_jobcard) * 100

In [ ]:
print(str(overallMatchSHG)+"%") # final match over SHG data percentage

80.0%


In [ ]:
print(str(overallMatchJC)+"%") # final match over JobcardMerge percentage

0.6056018168054504%
